In [1]:
import pandas as pd
import torch
import torch.optim as optim
import numpy as np
import itertools

from transformers import BertTokenizer
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelBinarizer
from BrandClassifier import ThresholdClassifier
from DataPreprocess import BrandDataset

g:\Code\Python\GitHub\maboo\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('brand_data\\brand_data_4.csv')[['name', 'bra_整合']]
le= LabelBinarizer()
X = data['name']
y = le.fit_transform(data['bra_整合'])

In [7]:
device = torch.device('cuda')
max_length = 64
batch_size = 128
learning_rate = 5e-4
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

# train_data = data.sample(frac=0.8, random_state=87)
train_data = data
# val_data = data.drop(train_data.index).reset_index(drop=True)
# train_data = train_data.reset_index(drop=True)

train_dataset = BrandDataset(X, y, tokenizer=tokenizer, max_length=max_length)
# val_dataset = BrandDataset(val_data, tokenizer=tokenizer, max_length=max_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size)

model = ThresholdClassifier()
# model.load_state_dict(torch.load('saved_model/Brand_classifier_temp.ckpt'))
model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(params=model.parameters(), lr=learning_rate)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
def train(train_loader=None, val_loader=None, model=None, epochs=None, criterion=None, optimizer=None, l1_lambda=None, l2_lambda=0.001):
    
    scheduler_1 = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    scheduler_2 = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=7, factor=0.5)
    all_train_accs = []
    all_train_loss = []
    all_val_accs = []
    all_val_loss = []
    for epoch in itertools.count():
        
        model.train()
        train_loss = []
        train_accs = []
        
        for batch in train_loader:
            
            ids = batch['ids'].to(device, dtype=torch.long)
            mask = batch['mask'].to(device, dtype=torch.long)
            y = batch['targets'].to(device, dtype=torch.float32)
            
            logits = model(ids, mask)
            loss = criterion(logits, y)
            
            # L1 regularization with normalized l1
            if l1_lambda is not None:
                L1_regularization = sum(p.abs().sum() for p in model.parameters())
                param_num = sum(p.numel() for p in model.parameters())
                loss += (l1_lambda / param_num) * L1_regularization
            
            # L2 regularization with normalized l2
            if l2_lambda is not None:
                L2_regularization = sum(p.pow(2.0).sum() for p in model.parameters())
                # param_num = sum(p.numel() for p in model.parameters())
                loss += l2_lambda * L2_regularization
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            acc = float(((logits.argmax(dim=-1) == y.argmax(dim=-1)).sum()).detach().cpu() / y.shape[0])
            train_loss.append(loss.item())
            train_accs.append(acc)
        
        train_loss = sum(train_loss) / len(train_loss)
        train_acc = sum(train_accs) / len(train_accs)
        
        all_train_loss.append(train_loss)
        all_train_accs.append(train_acc)
        
        scheduler_1.step()
        scheduler_2.step(train_loss)
        
        if val_loader is not None:
            model.eval()

            valid_loss = []
            valid_accs = []
            
            for batch in val_loader:
                
                ids = batch['ids'].to(device, dtype=torch.long)
                mask = batch['mask'].to(device, dtype=torch.long)
                y = batch['targets'].to(device, dtype=torch.float32)
                
                with torch.no_grad():
                    
                    logits = model(ids, mask)

                    acc = float(((logits.argmax(dim=-1) == y.argmax(dim=-1)).sum()).detach().cpu() / y.shape[0])
                    valid_loss.append(loss.item())
                    valid_accs.append(acc)
            
            valid_loss = sum(valid_loss) / len(valid_loss)
            valid_acc = sum(valid_accs) / len(valid_accs)
            
            all_val_loss.append(valid_loss)
            all_val_accs.append(valid_acc)
            
            if epochs is not None:
                print(f'[ {epoch+1}/{epochs} ] | train_loss = {train_loss:.5f}, train_acc = {train_acc:.5f}, val_loss = {valid_loss:.5f}, val_acc = {valid_acc:.5f}')
            else:
                print(f'[ {epoch+1} ] | train_loss = {train_loss:.5f}, train_acc = {train_acc:.5f}, val_loss = {valid_loss:.5f}, val_acc = {valid_acc:.5f}')
                if epoch+1 == epochs:
                    return model, [[all_train_accs, all_train_loss], [all_val_accs, all_val_loss]]
            
            if np.mean(all_train_accs[-15:]) > train_acc:
                print('Accuracy no longer increase, stop training!')
                return model, [[all_train_accs, all_train_loss], [all_val_accs, all_val_loss]]
        
        else:
            if epochs is not None:
                print(f'[ {epoch+1}/{epochs} ] | train_loss = {train_loss:.5f}, train_acc = {train_acc:.5f}')
            else:
                print(f'[ {epoch+1} ] | train_loss = {train_loss:.5f}, train_acc = {train_acc:.5f}')
                if epoch+1 == epochs:
                    return model, [all_train_accs, all_train_loss]
        
        if train_acc >= max(all_train_accs):
            torch.save(model.state_dict(), 'saved_model/Brand_classifier_temp.ckpt')
                
        if (len(all_train_accs) >= 50) and (np.mean(all_train_loss[-25:]) > train_loss):
            print('Loss no longer decrease, stop training!')
            return model, [all_train_accs, all_train_loss]

In [9]:
model, history = train(train_loader=train_loader, model=model, epochs=None, criterion=criterion, optimizer=optimizer)

[ 1 ] | train_loss = 187.63061, train_acc = 0.50438
[ 2 ] | train_loss = 187.62740, train_acc = 0.50095
[ 3 ] | train_loss = 187.63649, train_acc = 0.49912
[ 4 ] | train_loss = 187.62012, train_acc = 0.50684
[ 5 ] | train_loss = 187.63605, train_acc = 0.49773
[ 6 ] | train_loss = 187.61873, train_acc = 0.50803
[ 7 ] | train_loss = 187.62910, train_acc = 0.50495
[ 8 ] | train_loss = 187.62612, train_acc = 0.50086
[ 9 ] | train_loss = 187.63074, train_acc = 0.50196
[ 10 ] | train_loss = 187.61251, train_acc = 0.50854
[ 11 ] | train_loss = 187.63166, train_acc = 0.50123
[ 12 ] | train_loss = 187.63934, train_acc = 0.50139
[ 13 ] | train_loss = 187.63318, train_acc = 0.50141
[ 14 ] | train_loss = 187.62060, train_acc = 0.50827
[ 15 ] | train_loss = 187.62006, train_acc = 0.50731
[ 16 ] | train_loss = 187.63395, train_acc = 0.49740
[ 17 ] | train_loss = 187.62672, train_acc = 0.50816
[ 18 ] | train_loss = 187.62928, train_acc = 0.50381
[ 19 ] | train_loss = 187.61811, train_acc = 0.50405
[ 

KeyboardInterrupt: 

In [ ]:
# torch.save(model, 'saved_model/whole_Brand_classifier.ckpt')